# Herkunftsländer der Rassen in den Datensatz einfügen

#### Imports

In [34]:
import pandas as pd
import requests
import time
from tqdm import tqdm
import numpy as np

#### Datensatz laden

In [35]:
df = pd.read_csv("dogs-ranking-dataset.csv")
df.head(1)

,Breed,type,score,popularity ranking,size,intelligence,congential ailments,score for kids,size.1,$LIFETIME COST,INTELLIGENCE RANK,INTELLIGENCE %,LONGEVITY(YEARS),NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS,PURCHASE PRICE,FOOD COSTS PER YEAR,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN
0,Border Terrier,terrier,3.61,61,1,Above average,none,4.99,small,"$22,638",30,70%,14.0,0,none,$833,$324,Once a week,1


### Währungsspalten umkonvertieren

In [ ]:
def convert_money_columns(df, currency_pattern=r"\$"):
    # Spaltennamen mit $ erkennen
    name_mask = df.columns.str.contains(currency_pattern)

    # Spaltenwerte mit $ erkennen (nur object-Spalten)
    object_cols = df.select_dtypes("object")
    value_mask_obj = object_cols.apply(lambda s: s.str.contains(currency_pattern, na=False)).any()

    # value_mask auf alle Spalten ausweiten (False bei anderen)
    value_mask_full = df.columns.isin(value_mask_obj[value_mask_obj].index)

    # Kombinieren
    money_cols = df.columns[name_mask | value_mask_full]

    # Umwandlung
    df[money_cols] = (
        df[money_cols]
        .apply(lambda s: (
            s.astype(str)
             .str.replace(r"[^\d.\-]", "", regex=True)
             .replace("", np.nan)
             .astype(float)
        ))
    )
    return df


In [37]:
df = convert_money_columns(df)
df

,Breed,type,score,popularity ranking,size,intelligence,congential ailments,score for kids,size.1,$LIFETIME COST,INTELLIGENCE RANK,INTELLIGENCE %,LONGEVITY(YEARS),NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS,PURCHASE PRICE,FOOD COSTS PER YEAR,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN
0,Border Terrier,terrier,3.61,61,1,Above average,none,4.99,small,22638.0,30,70%,14.00,0,none,833.0,324.0,Once a week,1
1,Cairn Terrier,terrier,3.53,48,1,Above average,"'lion jaw', heart problems",4.91,small,21992.0,35,61%,13.84,2,"'lion jaw', heart problems",435.0,324.0,Once a week,1
2,Siberian Husky,working,3.22,16,2,Average,none,4.72,medium,22049.0,45,45%,12.58,0,none,650.0,466.0,Once in a few weeks,1
3,Welsh Springer Spaniel,sporting,3.34,81,2,Above average,hip problems,4.71,medium,20224.0,31,69%,12.49,1,hip problems,750.0,324.0,Once a week,1
4,English Cocker Spaniel,sporting,3.33,51,2,Excellent,none,4.70,medium,18993.0,18,82%,11.66,0,none,800.0,324.0,Once a week,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Alaskan Malamute,working,1.82,47,3,Average,"hip problems, dwarfism",2.57,large,21986.0,50,36%,10.67,2,"hip problems, dwarfism",1210.0,710.0,Daily,2
83,Bloodhound,hound,1.66,42,3,Lowest,"fatal stomach bloat, skin problems",2.54,large,13824.0,75,7%,6.75,2,"fatal stomach bloat, skin problems",608.0,710.0,Once a week,2
84,Chow Chow,non-sporting,1.76,54,2,Lowest,"eye, hip problems",2.51,medium,15898.0,77,5%,9.01,2,"eye, hip problems",515.0,466.0,Daily,2
85,Akita,working,1.95,41,3,Average,hip problems,2.33,large,20994.0,54,31%,10.16,1,hip problems,1202.0,710.0,Once a week,3


#### Wikidata-ID (Q-ID) der jeweiligen Rasse suchen

In [38]:
def get_qid(breed):
    """Schritt 1: Hole die Wikidata-ID (Q-ID) der jeweiligen Rasse von Wikipedia."""
    print("API‑Titel, der rausgeht  →", breed)     # <— DEBUG-Zeile
    WIKI_API = "https://en.wikipedia.org/w/api.php"                 # Basis-URL Wikipedia-API
    params = {
        "action": "query",          # Query-Modul für Abfragen
        "format": "json",           # Antwort im JSON-Format
        "prop": "pageprops",        # Seiteneigenschaften
        "ppprop": "wikibase_item",  # von der Eigenschaften nur die Eigenschaft wikibase_item: Q-ID
        "titles": breed,            # Titel der Wikipediaseite, also jeweilige Rasse
        "redirects": 1              # Weiterleitung falls die gefundene Seite auf den Originalnamen verweist (z.B. bei veralteten Namen oder Spitznamen der Rasse)
    }
    response = requests.get(WIKI_API, params=params, timeout=15)    # Ergebnis der Anfrage an WIKI-API mit festgelegten Parametern und timeout=15(falls der Server nach 15sek nicht antwortet) in response speichern
    data = response.json()                                          # Umwandlung in ein Dictionary für leichtere Verarbeitung
    pages = data["query"]["pages"]                                  # Seitendaten aus der Anfrage speichern
    page = next(iter(pages.values()))                               # Trefferseite speichern
    return page.get("pageprops", {}).get("wikibase_item")           # Q-ID der gesuchten Rasse aus der Seite auslesen und ausgeben, falls fehlt None zurückgeben

In [39]:
# Test get_qid
test_breeds = ["Beagle", "German Shepherd", "Shiba Inu"]
for b in test_breeds:
    print(b, "→", get_qid(b))

API‑Titel, der rausgeht  → Beagle
Beagle → Q21102
API‑Titel, der rausgeht  → German Shepherd
German Shepherd → Q38280
API‑Titel, der rausgeht  → Shiba Inu
Shiba Inu → Q39315


#### Länder-Label holen

In [40]:
def get_country_label(qid):
    """Holt das englische oder deutsche Label für eine gegebene Q-ID."""
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"            # JSON-URL zum Wikidata-Eintrag der Q-ID
    try:
        response = requests.get(url, timeout=15)                                    # GET-Anfrage senden (max. 15 Sekunden warten)  
        response.raise_for_status()                                                 # Status abfragen
        data = response.json()                                                      # Umwandlung in ein Dictionary für leichtere Verarbeitung
        entity = data["entities"][qid]                                              # alle Infos über die Q-ID rausholen
        label = entity["labels"].get("de", {}).get("value") or \
                entity["labels"].get("en", {}).get("value")                         # erst versuchen das deutsche Label zu finden, sonst das englische                        
        return label                                                                # Herkunftsland zurückgeben
    except Exception as e:                                                          # wenn ein Fehler mit einer Q-ID auftritt:
        print(f"Fehler beim Nachladen von Label für {qid}: {e}")
        return None                                                                 # None zurückgeben

#### Herkunftsland per Q-ID aus Wikipedia ziehen

In [41]:
def get_countries_from_qid(qid):
    """Liefert das Herkunftsland (als String, z. B. "Germany/France")
    zu einer Wikidata‑Q‑ID. Durchsucht mehrere Properties, weil Wikidata
    Herkunft uneinheitlich speichert."""
    if not qid:                                                                     # falls keine Q-ID vorhanden ist
        return None                                                                 # Funktion abbrechen
    
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"            # JSON-URL zum Wikidata-Eintrag der Q-ID
    response = requests.get(url, timeout=15)                                        # GET-Anfrage senden (max. 15 Sekunden warten)           
    response.raise_for_status()                                                     # Status abfragen
    data = response.json()                                                          # Umwandlung in ein Dictionary für leichtere Verarbeitung

    claims = data["entities"][qid]["claims"]                                        # alle Aussagen ("claims") zum Wikidata-Objekt abrufen

    # --- Property-Liste: von "ideal" nach "zur Not" ---------------------
    # Erklärung:
    # P495  = country of origin         (genau das, was wir wollen)
    # P17   = country                   (oft bei Tieren benutzt)
    # P1532 = country for sport         (z. B. bei Jagdhunden)
    # P1001 = applies to jurisdiction   (manchmal Ersatz für Herkunft)
    # P27   = country of citizenship    (selten, aber als Fallback nützlich)

    props_to_check = ("P495", "P17", "P1532", "P1001","P27")

    country_labels = []                                                             # leere Liste für Länderbezeichnungen erstellen

    for prop in props_to_check:                                                     # Relevante Properties durchgehen
        if prop not in claims:                                                      # wenn die Property nicht vorhanden ist:
            print(f"Property {prop} nicht gefunden")
            continue                                                                # gehe zur nächsten Property

        for claim in claims[prop]:                                                  # jeden Aussage zur Property durchgehen
            try:
                country_id = claim["mainsnak"]["datavalue"]["value"]["id"]          # Q-ID des Landes extrahieren

                label = get_country_label(country_id)                               # Englisches Label des Landes lesen
                if label:
                    print(f"Gefundenes Land: {label} (QID: {country_id}, Property: {prop})")
                    country_labels.append(label)                                    # Label zur Liste hinzufügen
            except Exception:                                                       # falls das Label fehlt:
                print(f"Fehler beim Auslesen des Landes bei Property {prop}: {e}")
                continue                                                            # überspringen

        if country_labels:                                                          # wenn mindestens ein Land gefunden wurde:
            countries = "/".join(sorted(set(country_labels)))                       # Duplikate entfernen, alphabetisch sortieren, mit "/" verbinden
            return countries                                                        # Länder zurückgeben
            
    return None                                                                     # Falls keine Länder gefunden wurden, None zurückgeben

In [42]:
# Test get_countries_from_qid
qid = get_qid("Beagle")
print("QID:", qid)

country = get_countries_from_qid(qid)
print("Herkunftsland:", country)


API‑Titel, der rausgeht  → Beagle
QID: Q21102
Gefundenes Land: Vereinigtes Königreich (QID: Q145, Property: P495)
Herkunftsland: Vereinigtes Königreich


#### Herkunftsländer abfragen

In [43]:
origin_cache = {}                           # Cache: speichert bereits ermittelte Länder pro Rasse, um doppelte API‑Aufrufe zu vermeiden

def fetch_origin(breed):
    """Ermittelt das Herkunftsland einer Rasse (String oder None) und nutzt einen Cache, um Mehrfachanfragen zu vermeiden."""
    if breed in origin_cache:               # falls der Rassenname bereits in origin_cache vorhanden ist:
        return origin_cache[breed]          # gebe das dort gespeicherte Land zurück
    
    qid = get_qid(breed)                    # hole die Q-ID
    country = get_countries_from_qid(qid)   # hole das Herkunftsland

    origin_cache[breed] = country           # Cache aktualisieren (auch wenn country None ist)
    time.sleep(0.3)                         # kleine Pause um nicht zu viele Anfragen zu schicken
    return country                          # Herkunftsland (oder None) zurückgeben

#### Abfrage durchführen

In [44]:
tqdm.pandas()   # Ladebalken
df["origin_de"] = df["Breed"].progress_apply(fetch_origin)     # Herkunftsland der Rasse in die neue Spalte origin_country einfügen

  0%|          | 0/87 [00:00<?, ?it/s]

API‑Titel, der rausgeht  → Border Terrier
Gefundenes Land: Vereinigtes Königreich (QID: Q145, Property: P495)


  2%|▏         | 2/87 [00:01<01:12,  1.17it/s]

API‑Titel, der rausgeht  → Cairn Terrier
Gefundenes Land: Schottland (QID: Q22, Property: P495)


  3%|▎         | 3/87 [00:03<01:35,  1.14s/it]

...

 99%|█████████▉| 86/87 [02:17<00:01,  1.61s/it]

API‑Titel, der rausgeht  → Akita
Property P495 nicht gefunden
Property P17 nicht gefunden
Property P1532 nicht gefunden
Property P1001 nicht gefunden
Property P27 nicht gefunden


100%|██████████| 87/87 [02:18<00:00,  1.33s/it]

API‑Titel, der rausgeht  → Bulldog
Gefundenes Land: Vereinigtes Königreich (QID: Q145, Property: P495)


100%|██████████| 87/87 [02:20<00:00,  1.61s/it]


#### neuen Datensatz speichern

In [45]:
df.to_csv("dogs-ranking-dataset-with-origin.csv", index=False)

#### Fehlende Werte finden

In [46]:
# neue CSV laden
df = pd.read_csv("dogs-ranking-dataset-with-origin.csv")

# fehlende Werte in Spalte Herkunftsland finden
fehlende_herkunft = df[df["origin_de"].isna()]

# fehlende Werte in einer Datei speichern zum Nachtragen
# auskommentiert, damit meine händischen Änderungen nicht verloren gehen, sollte man das ganze Skript ausführen
#fehlende_herkunft.to_csv("missing_origin.csv", index=False)

##### fehlende Werte händisch ergänzt

#### beide CSVs laden

In [47]:
# Original-CSV laden
df_original = pd.read_csv("dogs-ranking-dataset-with-origin.csv")

# Ergänzungs-CSV laden
df_ergänzung = pd.read_csv("missing_origin.csv")

#### Daten zusammenführen

In [48]:
# beide DataFrames anhand der Rasse verbinden (left merge, damit alle Original-Daten bleiben)
df_merged = pd.merge(df_original, df_ergänzung[['Breed', 'origin_de']], on="Breed", how="left")

# Kombinieren, falls eine Spalte leer ist
df_merged['origin_de'] = df_merged['origin_de_x'].fillna(df_merged['origin_de_y'])

# Danach alte Spalten löschen
df_merged = df_merged.drop(columns=['origin_de_x', 'origin_de_y'])

#### Ergebnis speichern

In [49]:
df_merged.to_csv("final_dataset.csv", index=False)

#### deutsche Rassennamen hinzufügen

In [50]:
# englische Rassennamen rausfiltern
breeds = list(df_merged["Breed"])
breeds

['Border Terrier',
 'Cairn Terrier',
 'Siberian Husky',
 'Welsh Springer Spaniel',
 'English Cocker Spaniel',
 'Cocker Spaniel',
 'Lhasa Apso',
 'English Springer Spaniel',
 'Shetland Sheepdog',
 'West Highland White Terrier',
 'Brittany',
 'German Shorthaired Pointer',
 'Pointer',
 'Tibetan Spaniel',
 'Labrador Retriever',
 'Bichon Frise',
 'Irish Setter',
 'Samoyed',
 'Shih Tzu',
 'Golden Retriever',
 'Chesapeake Bay Retriever',
 'Papillon',
 'Gordon Setter',
 'English Setter',
 'Pug',
 'Affenpinscher',
 'Miniature Schnauzer',
 'Beagle',
 'Border Collie',
 'Australian Terrier',
 'Whippet',
 'Boston Terrier',
 'Briard',
 'Bedlington Terrier',
 'Cavalier King Charles Spaniel',
 'Dalmatian',
 'Flat-Coated Retriever',
 'Belgian Tervuren',
 'Basset Hound',
 'Poodle',
 'Staffordshire Bull Terrier',
 'Bouvier des Flandres',
 'Pembroke Welsh Corgi',
 'Clumber Spaniel',
 'Pomeranian',
 'Australian Shepherd',
 'Pharaoh Hound',
 'Dandie Dinmont Terrier',
 'Greyhound',
 'Saluki',
 'Australian Ca

In [51]:
# Übersetzung von ChatGPT
breed_translation = {
    'Border Terrier': 'Border Terrier',
    'Cairn Terrier': 'Cairn Terrier',
    'Siberian Husky': 'Sibirischer Husky',
    'Welsh Springer Spaniel': 'Walisischer Springer Spaniel',
    'English Cocker Spaniel': 'Englischer Cocker Spaniel',
    'Cocker Spaniel': 'Cocker Spaniel',
    'Lhasa Apso': 'Lhasa Apso',
    'English Springer Spaniel': 'Englischer Springer Spaniel',
    'Shetland Sheepdog': 'Shetland Sheepdog',
    'West Highland White Terrier': 'West Highland White Terrier',
    'Brittany': 'Epagneul Breton',
    'German Shorthaired Pointer': 'Deutsch Kurzhaar',
    'Pointer': 'Pointer',
    'Tibetan Spaniel': 'Tibetspaniel',
    'Labrador Retriever': 'Labrador Retriever',
    'Bichon Frise': 'Bichon Frisé',
    'Irish Setter': 'Irischer Setter',
    'Samoyed': 'Samojede',
    'Shih Tzu': 'Shih Tzu',
    'Golden Retriever': 'Golden Retriever',
    'Chesapeake Bay Retriever': 'Chesapeake Bay Retriever',
    'Papillon': 'Papillon',
    'Gordon Setter': 'Gordon Setter',
    'English Setter': 'Englischer Setter',
    'Pug': 'Mops',
    'Affenpinscher': 'Affenpinscher',
    'Miniature Schnauzer': 'Zwergschnauzer',
    'Beagle': 'Beagle',
    'Border Collie': 'Border Collie',
    'Australian Terrier': 'Australischer Terrier',
    'Whippet': 'Whippet',
    'Boston Terrier': 'Boston Terrier',
    'Briard': 'Briard',
    'Bedlington Terrier': 'Bedlington Terrier',
    'Cavalier King Charles Spaniel': 'Cavalier King Charles Spaniel',
    'Dalmatian': 'Dalmatiner',
    'Flat-Coated Retriever': 'Flat-Coated Retriever',
    'Belgian Tervuren': 'Belgischer Schäferhund (Tervueren)',
    'Basset Hound': 'Basset Hound',
    'Poodle': 'Pudel',
    'Staffordshire Bull Terrier': 'Staffordshire Bullterrier',
    'Bouvier des Flandres': 'Flandrischer Treibhund',
    'Pembroke Welsh Corgi': 'Pembroke Welsh Corgi',
    'Clumber Spaniel': 'Clumber Spaniel',
    'Pomeranian': 'Zwergspitz (Pomeranian)',
    'Australian Shepherd': 'Australian Shepherd',
    'Pharaoh Hound': 'Pharaonenhund',
    'Dandie Dinmont Terrier': 'Dandie Dinmont Terrier',
    'Greyhound': 'Greyhound',
    'Saluki': 'Saluki',
    'Australian Cattle Dog': 'Australischer Treibhund',
    'Tibetan Terrier': 'Tibet Terrier',
    'Norfolk Terrier': 'Norfolk Terrier',
    'Dachshund': 'Dackel (Teckel)',
    'Chihuahua': 'Chihuahua',
    'Doberman Pinscher': 'Dobermann',
    'English Toy Spaniel': 'Englischer Toy Spaniel',
    'Newfoundland': 'Neufundländer',
    'Basenji': 'Basenji',
    'Afghan Hound': 'Afghanischer Windhund',
    'Old English Sheepdog': 'Bobtail',
    'French Bulldog': 'Französische Bulldogge',
    'Bernese Mountain Dog': 'Berner Sennenhund',
    'Boxer': 'Boxer',
    'Brussels Griffon': 'Brüsseler Griffon',
    'Maltese': 'Malteser',
    'Giant Schnauzer': 'Riesenschnauzer',
    'Rottweiler': 'Rottweiler',
    'Yorkshire Terrier': 'Yorkshire Terrier',
    'Irish Wolfhound': 'Irischer Wolfshund',
    'Scottish Terrier': 'Scottish Terrier',
    'Bullmastiff': 'Bullmastiff',
    'German Shepherd': 'Deutscher Schäferhund',
    'Mastiff': 'Mastiff',
    'Great Dane': 'Deutsche Dogge',
    'Kerry Blue Terrier': 'Kerry Blue Terrier',
    'Italian Greyhound': 'Italienisches Windspiel',
    'Pekingese': 'Pekinese',
    'Rhodesian Ridgeback': 'Rhodesian Ridgeback',
    'Bull Terrier': 'Bullterrier',
    'Saint Bernard': 'Bernhardiner',
    'Borzoi': 'Russischer Windhund (Borzoi)',
    'Alaskan Malamute': 'Alaskan Malamute',
    'Bloodhound': 'Bluthund',
    'Chow Chow': 'Chow-Chow',
    'Akita': 'Akita',
    'Bulldog': 'Englische Bulldogge'
}

In [52]:
df_merged["Breed_de"] = df_merged["Breed"].map(breed_translation)
df_merged.head(5)

,Breed,type,score,popularity ranking,size,intelligence,congential ailments,score for kids,size.1,$LIFETIME COST,...,INTELLIGENCE %,LONGEVITY(YEARS),NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS,PURCHASE PRICE,FOOD COSTS PER YEAR,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN,origin_de,Breed_de
0,Border Terrier,terrier,3.61,61,1,Above average,none,4.99,small,22638.0,...,70%,14.00,0,none,833.0,324.0,Once a week,1,Vereinigtes Königreich,Border Terrier
1,Cairn Terrier,terrier,3.53,48,1,Above average,"'lion jaw', heart problems",4.91,small,21992.0,...,61%,13.84,2,"'lion jaw', heart problems",435.0,324.0,Once a week,1,Schottland,Cairn Terrier
2,Siberian Husky,working,3.22,16,2,Average,none,4.72,medium,22049.0,...,45%,12.58,0,none,650.0,466.0,Once in a few weeks,1,Sibirien,Sibirischer Husky
3,Welsh Springer Spaniel,sporting,3.34,81,2,Above average,hip problems,4.71,medium,20224.0,...,69%,12.49,1,hip problems,750.0,324.0,Once a week,1,Wales,Walisischer Springer Spaniel
4,English Cocker Spaniel,sporting,3.33,51,2,Excellent,none,4.70,medium,18993.0,...,82%,11.66,0,none,800.0,324.0,Once a week,1,England,Englischer Cocker Spaniel


In [53]:
df_merged.to_csv("final_dataset.csv", index=False)

In [54]:
# type filtern zum übersetzen
type = list(df_merged["type"].unique())
type

['terrier', 'working', 'sporting', 'non-sporting', 'herding', 'toy', 'hound']

In [55]:
# Übersetzung der Rassetypen
type_translation = {
    'terrier': 'Terrier',
    'working': 'Gebrauchshund',
    'sporting': 'Jagdhund',
    'non-sporting': 'Begleithund',
    'herding': 'Hütehund',
    'toy': 'Zwerghund',
    'hound': 'Laufhund'
}

In [56]:
# neue Spalte mit deutschen type erstellen
df_merged["type_de"] = df_merged["type"].map(type_translation)
df_merged.head(1)

,Breed,type,score,popularity ranking,size,intelligence,congential ailments,score for kids,size.1,$LIFETIME COST,...,LONGEVITY(YEARS),NUMBER OF GENETIC AILMENTS,GENETIC AILMENTS,PURCHASE PRICE,FOOD COSTS PER YEAR,GROOMING FREQUNCY,SUITABILITY FOR CHILDREN,origin_de,Breed_de,type_de
0,Border Terrier,terrier,3.61,61,1,Above average,none,4.99,small,22638.0,...,14.0,0,none,833.0,324.0,Once a week,1,Vereinigtes Königreich,Border Terrier,Terrier


In [57]:
df_merged.to_csv("final_dataset.csv", index=False)